In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [14]:
def train(model, device, train_loader, optimizer, epoch,log_interval=10,dry_run=0):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [15]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
use_cuda = torch.cuda.is_available()

In [17]:
if torch.cuda.is_available():
         print("Trianing on a GPU")
else:
         print("Training on CPUs")

Trianing on a GPU


In [18]:
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")


In [19]:
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

In [20]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('data', train=False,
                      transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [21]:
model = Net().to(device)

In [22]:
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [23]:
for epoch in range(1, 5 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298651
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.203002
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.996240
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.580079
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.437692
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.485486
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.218080
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.602196
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.226749
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.133603
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.247851
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.249338
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.289733
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.213687
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.399869
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.159566
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.229066
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.210658
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.148199
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.175460
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.079567
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.037852
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.057459
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.101105
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.067817
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.051556
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.162108
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.091013
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.052441
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.020727
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.119848
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.050991
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.020959
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.322613
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.114649
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.025341
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.043930
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.154266
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.021891
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.065128


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.015137
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.021808
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.128043
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.276391
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.010889
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.052920
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.013295
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.035989
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.025480
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.003984
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.040407
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.011854
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.085240
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.001859
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.004060
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.062388
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.143341
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.002858
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.019213
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.010962


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.023811
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.035762
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.121069
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.005529
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.057201
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.027373
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.003319
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.004524
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.087071
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.035522
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.049415
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.012055
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.134308
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.003169
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.013171
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.011099
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.006228
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.001670
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.009620
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.003286


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.004718
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.001302
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.113978
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.080220

Test set: Average loss: 0.0263, Accuracy: 9921/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.094645
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.002331
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.001630
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.001572
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.014127
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.003517
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.002462
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.018041
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.006328
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.080510
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.025081
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.067494
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001350
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.022416
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.001878
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.029965
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.047414
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.006389
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.003694
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.001890
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.004798
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.059677
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.037858
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.006175
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.017750
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003092
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.004519
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.005381
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.170292
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.108676
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.010730
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.098157
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.027076
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.091684
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.009333
